In [9]:
import GA as ga
import pickle
import sklearn
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
from sklearn.datasets import fetch_california_housing
housing = fetch_california_housing()

In [3]:
df_data = pd.DataFrame(housing["data"])
df_data["target"] = housing["target"]

In [4]:
df_data

,0,1,2,3,4,5,6,7,target
0,8.3252,41.0,6.984127,1.023810,322.0,2.555556,37.88,-122.23,4.526
1,8.3014,21.0,6.238137,0.971880,2401.0,2.109842,37.86,-122.22,3.585
2,7.2574,52.0,8.288136,1.073446,496.0,2.802260,37.85,-122.24,3.521
3,5.6431,52.0,5.817352,1.073059,558.0,2.547945,37.85,-122.25,3.413
4,3.8462,52.0,6.281853,1.081081,565.0,2.181467,37.85,-122.25,3.422
...,...,...,...,...,...,...,...,...,...
20635,1.5603,25.0,5.045455,1.133333,845.0,2.560606,39.48,-121.09,0.781
20636,2.5568,18.0,6.114035,1.315789,356.0,3.122807,39.49,-121.21,0.771
20637,1.7000,17.0,5.205543,1.120092,1007.0,2.325635,39.43,-121.22,0.923
20638,1.8672,18.0,5.329513,1.171920,741.0,2.123209,39.43,-121.32,0.847


In [5]:
dic_natural_partition = {ix: [ix] for ix in df_data.index}

In [6]:
drop_cols = []

keys_natural_partition = sorted(dic_natural_partition.keys())
size_natural_partition = len(keys_natural_partition)
num_blocks = 4

set_all_indexs = set()
for k, v in dic_natural_partition.items():
    set_all_indexs = set_all_indexs.union(set(v))

y_col = "target"
X, y = df_data.loc[set_all_indexs].drop(columns=[y_col] + drop_cols), df_data.loc[set_all_indexs][y_col]
print(y.shape)
name_cols = np.array(list(X.columns))
print(list(name_cols))
num_cols = len(name_cols)
print(num_cols)
pmf_cols = ga.get_pmf_cols(X, y)
print(pmf_cols)

cols_selector_0, R2_generation_0 = np.zeros(num_cols, dtype=int), 0
cols_selector_0[[0]] = 1

evaluation_stratified_block_selector_0 = None
list_e_nb_fold_0 = None
R2_per_e_0 = None
candidates_0 = None
candidates_mutation_0 = None
mutation_cross_over_0 = {"0": cols_selector_0}

cols_selector_generations = [cols_selector_0]
evaluation_stratified_block_selector_generations = [evaluation_stratified_block_selector_0]
list_e_nb_fold_generations = [list_e_nb_fold_0]
R2_generations = [R2_generation_0]
R2_per_e_generations = [R2_per_e_0]
candidates_generations = [candidates_0]
candidates_mutation_generations = [candidates_mutation_0]
mutation_cross_over_generations = [mutation_cross_over_0]

cols_selector_n = cols_selector_0
evaluation_stratified_block_selector_n = evaluation_stratified_block_selector_0
list_e_nb_fold_n = list_e_nb_fold_0
R2_generation_n = R2_generation_0
R2_per_e_n = R2_per_e_0
candidates_n = candidates_0
candidates_mutation_n = candidates_mutation_0
mutation_cross_over_n = mutation_cross_over_0

N = 1 # Number of generations
while N<5:
    print(N)
    print("get_evaluation_seed")
    evaluation_stratified_block_selector_n, list_e_nb_fold_n = ga.get_evaluation_seed(size_natural_partition, num_blocks,
                                                                                   set_all_indexs,
                                                                                   dic_natural_partition,
                                                                                   keys_natural_partition, num_evaluation=25)
    print("get_mutation_cross_over_evaluation_metrics")
    mutation_cross_over_evaluation_metrics = ga.get_mutation_cross_over_evaluation_metrics(mutation_cross_over_n,
                                                                                        list_e_nb_fold_n, X, y,
                                                                                        name_cols, m="")
    print("mutation_cross_over_n")
    cols_selector_n = mutation_cross_over_n[mutation_cross_over_evaluation_metrics[0][0]]
    print("mutation_cross_over_evaluation_metrics")
    R2_generation_n, R2_per_e_n = mutation_cross_over_evaluation_metrics[0][1]
    print(R2_generation_n)
    print("get_candidates")
    candidates_n = ga.get_candidates(R2_per_e_n)
    print("get_candidates_mutation")
    candidates_mutation_n = ga.get_candidates_mutation(candidates_n, cols_selector_n, X, y, list_e_nb_fold_n, name_cols, pmf_cols)
    print("get_mutation_cross_over")
    mutation_cross_over_n = ga.get_mutation_cross_over(candidates_mutation_n, cols_selector_n)

    cols_selector_generations.append(cols_selector_n)
    evaluation_stratified_block_selector_generations.append(evaluation_stratified_block_selector_n)
    list_e_nb_fold_generations.append(list_e_nb_fold_n)
    R2_generations.append(R2_generation_n)
    R2_per_e_generations.append(R2_per_e_n)
    candidates_generations.append(candidates_n)
    candidates_mutation_generations.append(candidates_mutation_n)
    mutation_cross_over_generations.append(mutation_cross_over_n)

    o = {"cols_selector_generations": cols_selector_generations,
         "evaluation_stratified_block_selector_generations": evaluation_stratified_block_selector_generations,
         "list_e_nb_fold_generations": list_e_nb_fold_generations,
         "R2_generations": R2_generations,
         "R2_per_e_generations": R2_per_e_generations,
         "candidates_generations": candidates_generations,
         "candidates_mutation_generations": candidates_mutation_generations,
         "mutation_cross_over_generations": mutation_cross_over_generations}

#     pickle.dump(o, open(f"step_{N}.pickle", "wb"))
    N += 1

C:\Users\felip\AppData\Local\Temp/ipykernel_23676/23457276.py:12: FutureWarning: Passing a set as an indexer is deprecated and will raise in a future version. Use a list instead.
  X, y = df_data.loc[set_all_indexs].drop(columns=[y_col] + drop_cols), df_data.loc[set_all_indexs][y_col]
C:\Users\felip\AppData\Local\Temp/ipykernel_23676/23457276.py:12: FutureWarning: Passing a set as an indexer is deprecated and will raise in a future version. Use a list instead.
  X, y = df_data.loc[set_all_indexs].drop(columns=[y_col] + drop_cols), df_data.loc[set_all_indexs][y_col]


(20640,)
[0, 1, 2, 3, 4, 5, 6, 7]
8
[0.6880752079585524, 0.10562341249321056, 0.15194828974145708, 0.046700512969486775, 0.024649678888895077, 0.0237374129561344, 0.14416027687465738, 0.04596661511798007]
1.2308614070003734
[0.63819595 0.07866377 0.12316575 0.0220595  0.00087637 0.
 0.1156842  0.02135448]
1
get_evaluation_seed
get_mutation_cross_over_evaluation_metrics
0
0.4732044636673583
mutation_cross_over_n
mutation_cross_over_evaluation_metrics
0.4732044636673583
get_candidates
get_candidates_mutation
best
insert_mutation
0.07866376621536017 [0.5090023393640667, 1, 4, 1]
0.1231657486593964 [0.47789373568611426, 2, 4, 2]
0.02205949663241766 [0.4732457301875715, 3, 4, 3]
0.0008763680539605674 [0.47417847476351677, 4, 4, 4]
0.0 [0.47078973233773835, 5, 4, 5]
0.11568419592877655 [0.48142818578137375, 6, 4, 6]
0.021354477833804363 [0.4746715610193414, 7, 4, 7]
Insert: 1 0.5090023393640667
0.1231657486593964 [0.5110298126747563, 2, 4, 2]
0.02205949663241766 [0.5089442263759005, 3, 4, 3]

0.6027145417814033
mutation_cross_over_n
mutation_cross_over_evaluation_metrics
0.6027145417814033
get_candidates
get_candidates_mutation
best
insert_mutation
0.0 [0.6050416584100322, 5, 4, 5]
Insert: 5 0.6050416584100322
delete_mutation
0.6381959466762842 [0.397530041984476, 0, 4, 0]
0.07866376621536017 [0.5966408441164963, 1, 4, 1]
0.1231657486593964 [0.599043723996358, 2, 4, 2]
0.02205949663241766 [0.5953480354437681, 3, 4, 3]
0.0008763680539605674 [0.6050347452989396, 4, 4, 4]
0.0 [0.6039986195639274, 5, 4, 5]
0.11568419592877655 [0.5389571765227975, 6, 4, 6]
0.021354477833804363 [0.5410628191684911, 7, 4, 7]
insert_mutation
worst
insert_mutation
0.0 [0.6024603119684677, 5, 4, 5]
Insert: 5 0.6024603119684677
delete_mutation
0.6381959466762842 [0.3967449283958918, 0, 4, 0]
0.07866376621536017 [0.5941107895630354, 1, 4, 1]
0.1231657486593964 [0.5978295984313853, 2, 4, 2]
0.02205949663241766 [0.5951187331433949, 3, 4, 3]
0.0008763680539605674 [0.6024481010602556, 4, 4, 4]
0.0 [0.60145

In [18]:
for g in range(N):
    print(f""" Generation-{g}
    R2: {np.round(R2_generations[g], 3)}, col_selector: {cols_selector_generations[g]}
    """)

 Generation-0
    R2: 0, col_selector: [1 0 0 0 0 0 0 0]
    
 Generation-1
    R2: 0.473, col_selector: [1 0 0 0 0 0 0 0]
    
 Generation-2
    R2: 0.603, col_selector: [1 1 1 1 1 0 1 1]
    
 Generation-3
    R2: 0.603, col_selector: [1 1 1 1 1 0 1 1]
    
 Generation-4
    R2: 0.603, col_selector: [1 1 1 1 1 0 1 1]
    
